In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Input, Flatten, Dropout
from tensorflow.keras.layers import concatenate, Activation
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import to_categorical
import os
import numpy as np
import math

In [2]:
batch_size = 32
epochs = 200
data_augmentation = False # 단순 구현임으로 augmentation 제외

In [3]:
num_calsses = 10
num_dense_blocks = 3
use_max_pool = False

In [4]:
growth_rate = 12
depth = 100
num_bottleneck_layers = (depth-4)//(2*num_dense_blocks)

In [5]:
num_filters_bef_dense_block= 2*growth_rate
compression_factor = 0.5

In [6]:
#load data
(x_train,y_train),(x_test,y_test) = cifar10.load_data()

In [7]:
input_shape = x_train.shape[1:]

In [10]:
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

In [11]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [12]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch >180:
        lr *=0.5e-3
    elif epoch >160:
        lr *=1e-3
    elif epoch >120:
        lr*=1e-2
    elif epoch >80:
        lr *=1e-1
        
    return lr

In [13]:
inputs = Input(shape = input_shape)
x= BatchNormalization()(inputs)
x = Activation('relu')(x)
x = Conv2D(
    num_filters_bef_dense_block,
    kernel_size = 3,
    padding = 'same',
    kernel_initializer = 'he_normal'
)(x)
x = concatenate([inputs,x])

In [15]:
for i in range (num_dense_blocks):
    
    for j in range(num_bottleneck_layers):
        y = BatchNormalization()(x)
        y = Activation('relu')(y)
        y = Conv2D(
            4*growth_rate,
            kernel_size =1,
            padding = 'same',
            kernel_initializer = 'he_normal'
        )(y)
        if not data_augmentation:
            y = Dropout(0.2)(y)
        
        y = BatchNormalization()(y)
        y = Activation('relu')(y)
        y = Conv2D(
            growth_rate,
            kernel_size = 3,
            padding = 'same',
            kernel_initializer = 'he_normal'
        )(y)
        
        if not data_augmentation:
            y = Dropout(0.2)(y)
        
        x = concatenate([x,y])
        
    if i == num_dense_blocks -1:
        continue
    
    num_filters_bef_dense_block+=num_bottleneck_layers * growth_rate
    num_filters_bef_dense_block = int(num_filters_bef_dense_block*compression_factor)
    
    y = BatchNormalization()(x)
    y = Conv2D(
        num_filters_bef_dense_block,
        kernel_size = 1,
        padding = 'same',
        kernel_initializer = 'he_normal'
    )(y)
    
    if not data_augmentation:
        y = Dropout(0.2)(y)
    x = AveragePooling2D()(y)

In [16]:
x = AveragePooling2D(pool_size = 8)(x)
y = Flatten()(x)
outputs = Dense(
    num_calsses,
    kernel_initializer ='he_normal',
    activation = 'softmax'
)(y)

In [17]:
model = Model(inputs = inputs,outputs = outputs)
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = RMSprop(1e-3),
    metrics = ['acc']
)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 3)    12          input_1[0][0]                    
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 3)    0           batch_normalization[0][0]        
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 24)   672         activation[0][0]                 
______________________________________________________________________________________________

In [18]:
save_dir = os.path.join(os.getcwd(),'save_models')

In [19]:
model_name = 'cifar10_densenet_model.{epoch:02d}.h5'

In [20]:
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir,model_name)

In [22]:
checkpoint = ModelCheckpoint(
    filepath = filepath,
    monitor ='val_acc',
    verbose = 1,
    save_best_only = True
)

In [24]:
lr_scheduler = LearningRateScheduler(lr_schedule)

In [25]:
lr_reducer = ReduceLROnPlateau(
    factor = np.sqrt(0.1),
    cooldown = 0,
    patience=5,
    min_lr = 0.5e-6
)

In [26]:
callbacks = [checkpoint,lr_reducer,lr_scheduler]

In [29]:
model.fit(
    x_train,
    y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = (x_test,y_test),
    shuffle = True,
    callbacks = callbacks
    
)

Epoch 1/200


KeyboardInterrupt: 